# Payroll Report using Deputy and Airtable
### by Alejandra Ramos de Vega

In [1]:
import pandas as pd
import numpy as np
import math

In [11]:
df_m1=pd.read_csv('Payroll_Deputy_Report.csv')
df_m1.insert(15,'Status', ' ')

In [27]:
# 1. Sick Leave with Doctors Notice >>>>> 2
# 4. Annual Leave (Vacation) >>>>>> 3
# Working hours >>>>> 1

i=0
for i in range(len(df_m1['Employee Export Code'])):
    if df_m1['Leave'][i]=='1. Sick Leave with Doctors Notice':
        df_m1['Status'][i]=2
    else:  
        if df_m1['Leave'][i]=='4. Annual Leave (Vacation)':
                df_m1['Status'][i]=3
        else:
            df_m1['Status'][i]=1
    i=i+1           

/var/folders/cz/mrz_s61n5r1c3_x70xssn2gh0000gn/T/ipykernel_24911/2716840622.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m1['Status'][i]=3
/var/folders/cz/mrz_s61n5r1c3_x70xssn2gh0000gn/T/ipykernel_24911/2716840622.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m1['Status'][i]=1
/var/folders/cz/mrz_s61n5r1c3_x70xssn2gh0000gn/T/ipykernel_24911/2716840622.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m1['Sta

---------

In [29]:
#Status = 1 -----> working hours of the 1st month 
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
hours_m1=pysqldf('SELECT("Employee Export Code"),("Employee Name"),sum("Total Hours"),("Status") from df_m1 where Status=1 group by ("Employee Export Code")')

In [30]:
#Status = 3 -----> vacation hours of the 1st month 
pysqldf = lambda q: sqldf(q, globals())
hours_m1_vac=pysqldf('SELECT("Employee Export Code"),("Employee Name"),sum("Total Hours"),("Status") from df_m1 where Status=3 group by ("Employee Export Code")')

In [31]:
#Status = 2 -----> sick hours of the 1st month 
pysqldf = lambda q: sqldf(q, globals())
hours_m1_sick=pysqldf('SELECT("Employee Export Code"),("Employee Name"),sum("Total Hours"),("Status") from df_m1 where Status=2 group by ("Employee Export Code")')

-------

### Unique Employees Name and Payroll ID

In [32]:
pysqldf = lambda q: sqldf(q, globals())
employees= pysqldf('SELECT DISTINCT("Employee Export Code"),("Employee Name") from df_m1')

In [33]:
employees

,Employee Export Code,Employee Name
0,ID1234567,Pedro Sanchez
1,ID23456,Rafael Suarez
2,ID67890,Veronica Perez
3,ID87654,Barry Pipe
4,ID75645342,Abby Chee


-----

### New Table Format

In [34]:
employees.insert(2, 'Worked hours July (Hours)', ' ')
employees.insert(3, 'Sick hours July (Hours)', ' ')
employees.insert(4, 'Holidays July (Hours)' , ' ')
employees.insert(5, 'Hourly wage (Euro/Hours)' , ' ')
employees.insert(6, 'Position' , ' ')
employees.insert(7, 'Store' , ' ')
employees.insert(8, 'Working Type' , ' ')


In [35]:
employees

,Employee Export Code,Employee Name,Worked hours July (Hours),Sick hours July (Hours),Holidays July (Hours),Hourly wage (Euro/Hours),Position,Store,Working Type
0,ID1234567,Pedro Sanchez,,,,,,,
1,ID23456,Rafael Suarez,,,,,,,
2,ID67890,Veronica Perez,,,,,,,
3,ID87654,Barry Pipe,,,,,,,
4,ID75645342,Abby Chee,,,,,,,


In [36]:
#Fill column Worked hours July (Hours) >>>>> with column "sum(Total Hours)" from hours_m1 

i=0
j=0
for i in range(len(employees['Employee Name'])):
    for j in range(len(hours_m1['Employee Name'])):
        if employees['Employee Export Code'][i]==hours_m1['Employee Export Code'][j]:
            if employees['Employee Name'][i]==hours_m1['Employee Name'][j]:
                employees['Worked hours July (Hours)'][i]=hours_m1['sum("Total Hours")'][j]
            else: j=j+1
    else:i=i+1        



In [37]:
#Fill column Holidays July (Hours) >>>>> with column "sum(Total Hours)" from hours_m1_vac 

i=0
j=0
for i in range(len(employees['Employee Name'])):
    for j in range(len(hours_m1_vac['Employee Name'])):
        if employees['Employee Export Code'][i]==hours_m1_vac['Employee Export Code'][j]:
            if employees['Employee Name'][i]==hours_m1_vac['Employee Name'][j]:
                employees['Holidays July (Hours)'][i]=hours_m1_vac['sum("Total Hours")'][j]
            else: j=j+1
    else:i=i+1 

In [38]:
#Fill column Sick hours July (Hours) >>>>> with column "sum(Total Hours)" from hours_m1_sick 

i=0
j=0
for i in range(len(employees['Employee Name'])):
    for j in range(len(hours_m1_sick['Employee Name'])):
        if employees['Employee Export Code'][i]==hours_m1_sick['Employee Export Code'][j]:
            if employees['Employee Name'][i]==hours_m1_sick['Employee Name'][j]:
                employees['Sick hours July (Hours)'][i]=hours_m1_sick['sum("Total Hours")'][j]
            else: j=j+1
    else:i=i+1 

In [39]:
employees

,Employee Export Code,Employee Name,Worked hours July (Hours),Sick hours July (Hours),Holidays July (Hours),Hourly wage (Euro/Hours),Position,Store,Working Type
0,ID1234567,Pedro Sanchez,33.0,16.0,120.0,,,,
1,ID23456,Rafael Suarez,35.72,10.0,,,,,
2,ID67890,Veronica Perez,39.89,,,,,,
3,ID87654,Barry Pipe,40.08,,,,,,
4,ID75645342,Abby Chee,83.58,,,,,,


<a href="https://www.linkedin.com/in/alejandradominique/">Alejandra Ramos de Vega</a> Data Analyst and Electrical Engineering